In [7]:
import numpy as np
import pandas as pd
import os

In [4]:
all_tickers = pd.read_excel('mapping.xlsx')[['Ticker', 'Market']]

In [5]:
all_tickers

,Ticker,Market
0,ARGT,Emerging
1,EWA,Developed
2,EWO,Developed
3,EWK,Developed
4,EWZ,Emerging
5,EWC,Developed
6,ECH,Emerging
7,MCHI,Emerging
8,ICOL,Emerging
9,EDEN,Developed


In [9]:
factor_to_analyze = 'mom_252'
file = 'ARGT.csv'

In [38]:
all_etf_combined_df = pd.DataFrame()

In [39]:
for file in os.listdir('forward return'):
    fr_df = pd.read_csv('forward return/'+file, index_col='Date')
    fr_df['Ticker'] = file[:-4]
    try:
        factor_df = pd.read_csv('factor data/'+factor_to_analyze+'/'+file[:-4]+'.csv', index_col='Date')
        combined_df = fr_df.join(factor_df).rename({factor_to_analyze: 'factor'}, axis=1)
        combined_df['factor'] = combined_df['factor'].fillna(method='ffill')
        all_etf_combined_df = pd.concat([all_etf_combined_df, combined_df.dropna().reset_index()], axis=0, ignore_index=True)
    except:
        continue

In [43]:
all_etf_combined_df.sort_values(['Date', 'Ticker']).set_index(['Date', 'Ticker'])

f_rtn    factor
Date       Ticker                    
1997-03-17 EWA    -0.016746  0.105820
           EWC    -0.060656  0.232323
           EWD    -0.010463  0.284069
           EWG    -0.023569  0.148196
           EWH    -0.008475  0.104524
...                     ...       ...
2019-11-27 QAT     0.003382 -0.040043
           THD    -0.010124  0.056992
           TUR     0.038314  0.058394
           UAE    -0.006603 -0.030583
           VOO     0.023134  0.200067

[164472 rows x 2 columns]

['ACWI.csv',
 'ARGT.csv',
 'ECH.csv',
 'EDEN.csv',
 'EFNL.csv',
 'EGPT.csv',
 'EIDO.csv',
 'EIRL.csv',
 'EIS.csv',
 'ENOR.csv',
 'ENZL.csv',
 'EPHE.csv',
 'EPOL.csv',
 'EPU.csv',
 'ERUS.csv',
 'EWA.csv',
 'EWC.csv',
 'EWD.csv',
 'EWG.csv',
 'EWH.csv',
 'EWI.csv',
 'EWJ.csv',
 'EWK.csv',
 'EWL.csv',
 'EWM.csv',
 'EWN.csv',
 'EWO.csv',
 'EWP.csv',
 'EWQ.csv',
 'EWS.csv',
 'EWT.csv',
 'EWU.csv',
 'EWW.csv',
 'EWY.csv',
 'EWZ.csv',
 'EZA.csv',
 'GREK.csv',
 'ICOL.csv',
 'INDA.csv',
 'KSA.csv',
 'MCHI.csv',
 'PAK.csv',
 'PGAL.csv',
 'QAT.csv',
 'THD.csv',
 'TUR.csv',
 'UAE.csv',
 'VOO.csv']